In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
pathtoJK = "../.."
sys.path.append(pathtoJK)
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from JKBio import GCPFunction as gcp
import numpy as np
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')

In [ ]:
bambucket = "gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_CRAM_files"
bamworkspace = "anvil-datastorage/AnVIL_GTEx_V8_hg38"
workspace = "broad-firecloud-ccle/DepMap_WGS_PON_hg38"
wgsworkspace = "broad-firecloud-ccle/DepMap_WGS_CN"

In [ ]:
wm = dm.WorkspaceManager(workspace)
datawm = dm.WorkspaceManager(bamworkspace)

In [ ]:
files = ! gsutil -u depmap-omics ls $bambucket

In [ ]:
len(files)

In [ ]:
crams = [f for f in files if f.endswith('.cram')]

In [ ]:
crams

In [ ]:
particip = datawm.get_participants()

In [ ]:
crams

In [ ]:
names = ['-'.join(c.split('/')[-1].split('.')[0].split('-')[:2]) for c in crams]
names

In [ ]:
len(set(names))

In [ ]:
particip = particip[~particip.wgs_cram_file.isna()]

In [ ]:
particip[particip.sex=='Male']

In [ ]:
malepart = particip[particip.sex=='Male']

In [ ]:
malepart = malepart.drop(columns=['has_rnaseq','ase_wasp_chrX_raw_counts',"ase_wasp_counts","has_genotype","wes_bam_file","wes_bam_index","ase_counts","ase_chrX_raw_counts"])

In [ ]:
malepart.index.name="sample_id"
malepart['participant_id']=malepart.index

In [ ]:
wm.upload_samples(malepart)

In [ ]:
wm.update_sample_set('GTEX_XY',malepart.index.tolist())

In [ ]:
wm.create_submission("CNV_Somatic_Panel_Workflow",'GTEX_XY')

In [ ]:
wgs = ! gsutil ls gs://cclebams/wgs/**.*

In [ ]:
len(wgs)

In [ ]:
sampletracker = 'https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE/edit#gid=1431277668'

In [ ]:
ccle_refsamples = sheets.get(sampletracker).sheets[0].to_frame()

In [ ]:
samples = ccle_refsamples[(ccle_refsamples.datatype=='wgs')]

In [ ]:
samples.columns

In [ ]:
samples = samples.set_index('cds_sample_id').drop(columns=['bam_qc', 'processing_qc',
       'crc32c_hash', '18q1', '18q2', '18q3', '18q4', '19q1', '19q2', '19q3',
       '19q4', '20q1', '20q2'])

In [ ]:
samples.index.name='sample_id'

In [ ]:
samples = samples.rename(columns={'patient_id':'participant_id'})

In [ ]:
wm = dm.WorkspaceManager(wgsworkspace)

In [ ]:
oldsamp = wm.get_samples()

In [ ]:
oldsamp.to_csv('../ccle_tasks/data/old_wgs_samples')

In [ ]:
wm.delete_sample(oldsamp.index.tol)

In [ ]:
oldsamp.columns

In [ ]:
oldsamp.loc['ccle_ACH-000045'][30:60]

In [ ]:
oldsamp.loc['ccle_ACH-000045'][-18:].index

In [ ]:
for val in ['rejected_mutations_fromliftover', 'readgroup_ubams',
       'duplication_metrics', 'ubam_file_list', 'output_sv_vcf',
       'candidate_indel_vcf_index', 'lifted_vcf', 'candidate_indel_vcf',
       'candidate_sv_vcf', 'analysis_ready_bam_index',
       'ubam_validation_reports', 'bqsr_report', 'cnn_filtered_vcf',
       'cnn_filtered_vcf_index', 'analysis_ready_bam_md5',
       'output_sv_vcf_index', 'analysis_ready_bam', 'candidate_sv_vcf_index']:
    samples[val] = None

In [ ]:
unique = samples[samples.version==1].index.tolist()
wm.update_sample_set('all_unique',unique)

In [ ]:
wm.enable_hound().upload_samples(samples)

In [ ]:
samples.columns

In [ ]:
samples[samples.arxspan_id=='ACH-000530']['sequencing_date']